In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat


cwd = os.getcwd()
cwd = os.path.join(cwd,'drive/My Drive/ProcessedData')

data = {}

folders = os.listdir(cwd)
for fruit in folders:
    records = []
    for record in os.listdir(os.path.join(cwd,fruit)):
        records.append(loadmat(os.path.join(cwd,fruit,record)))
    data[fruit] = records

In [3]:
print(data['Banana'][0]['vals'][:1])

[[[-1.26653463e-01 -1.83449358e-01  7.91000009e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]
  [-1.26653463e-01 -1.83449358e-01  7.91000009e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]
  [-1.24351025e-01 -1.83225185e-01  7.90000021e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]
  ...
  [ 2.39752516e-01 -1.77710652e-01  7.68000007e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]
  [ 2.39752516e-01 -1.77710652e-01  7.68000007e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]
  [ 2.39752516e-01 -1.77710652e-01  7.68000007e-01  2.55000000e+02
    2.55000000e+02  2.55000000e+02]]]


In [0]:
X,y = [],[]
for fruit_name in data:
    for fruit_sample in data[fruit]:
        X.append(fruit_sample['vals'])
        y.append(fruit_name)

In [5]:
y = np.asarray(y)
X = np.asarray(X)
print(y.shape)
print(X.shape)

(500,)
(500, 500, 500, 6)


In [0]:
X_new = X[:,:,:,[3,4,5,0,1,2]]

In [0]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
y = le.fit_transform(y)

y = to_categorical(y)

In [0]:
X_new[:,:,:,3:5] += 1
X_new[:,:,:,:3] /= 255
X_new[:,:,:,3] /= np.amax(X_new[:,:,:,3])
X_new[:,:,:,4] /= np.amax(X_new[:,:,:,4])
X_new[:,:,:,5] /= np.amax(X_new[:,:,:,5])

In [9]:
from sklearn.model_selection import train_test_split as split

X_train, X_test, y_train, y_test = split(X_new, y, test_size=.3)
X_train, X_val, y_train, y_val = split(X_train, y_train, test_size=0.3)
print(len(X_train))
print(len(X_val))
print(len(X_test))

print(len(y_train))
print(len(y_val))
print(len(y_test))

245
105
150
245
105
150


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D


model = Sequential()
model.add(Conv2D(40, 2, activation='relu', input_shape=(500,500,6)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model.add(Conv2D(20, 2, activation='relu', input_shape=(500,500,6)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 499, 499, 40)      1000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 249, 249, 40)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 248, 248, 20)      3220      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 124, 124, 20)      0         
_________________________________________________________________
flatten (Flatten)            (None, 307520)            0         
_________________________________________________________________
dense (Dense)                (None, 16)                4920336   
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1

In [13]:
hist = model.fit(X_train, y_train, batch_size=40, epochs=15, validation_data=(X_val, y_val))

Epoch 1/15
7/7 [==============================] - 8s 1s/step - loss: 16.1505 - accuracy: 0.1633 - val_loss: 1.6097 - val_accuracy: 0.1905
Epoch 2/15
7/7 [==============================] - 8s 1s/step - loss: 1.7191 - accuracy: 0.1673 - val_loss: 1.6098 - val_accuracy: 0.2000
Epoch 3/15
7/7 [==============================] - 8s 1s/step - loss: 1.6093 - accuracy: 0.2122 - val_loss: 1.6098 - val_accuracy: 0.2000
Epoch 4/15
7/7 [==============================] - 8s 1s/step - loss: 1.6097 - accuracy: 0.2122 - val_loss: 1.6099 - val_accuracy: 0.2000
Epoch 5/15
7/7 [==============================] - 8s 1s/step - loss: 1.6091 - accuracy: 0.2122 - val_loss: 1.6100 - val_accuracy: 0.2000
Epoch 6/15
7/7 [==============================] - 8s 1s/step - loss: 1.6096 - accuracy: 0.2122 - val_loss: 1.6100 - val_accuracy: 0.2000
Epoch 7/15
7/7 [==============================] - 8s 1s/step - loss: 1.6085 - accuracy: 0.2122 - val_loss: 1.6101 - val_accuracy: 0.2000
Epoch 8/15
7/7 [========================

In [14]:
model.evaluate(X_test, y_test)[1]

5/5 [==============================] - 1s 104ms/step - loss: 1.6103 - accuracy: 0.1800


0.18000000715255737